In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi
from importlib import reload

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

# np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

fileR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_enant_scan25step_moire.root")
fileS = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_enant_scan35step_moire.root")
# fileR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_enant_scan25step_moire_lim.root")
# fileS = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_enant_scan35step_moire_lim.root")

PATH_CH = ["CH11-2-3_m2x-m29/","CH9-10_m1x-m42/"]
PATH_CH_opt = 1

#manual selection..
# PATH_name = str(PATH_CH[PATH_CH_opt]).split("_")[0]+"_lim"
PATH_name = str(PATH_CH[PATH_CH_opt]).split("_")[0]
    

DIR = ["map_AVG_ph_En/","map_ScanEn_SHORT/","map_ScanEn_LONG/"]
EN = ["NO_gate","550","250","20"]
EN_opt = 2
# the next layer is L or R, selected later
ID_AVG=["PECD_el(theta)","PECD_el(theta)_redPHI"]
ID_SHORT=["cos(theta)_vs_pEn","cos(theta)_vs_pEn_pol","cos(theta)_vs_pEn_redPHI","cos(theta)_vs_pEn_pol_redPHI"]
ID_PECD=["PECD_el(theta)_@533eV_MINUS","PECD_el(theta)_@534eV_PLUS","PECD_el(theta)_@533eV_MINUS_redPHI","PECD_el(theta)_@534eV_PLUS_redPHI"]

#both ennatiomer (different en length)
loc_avg="angular_distr_el/"+PATH_CH[PATH_CH_opt]+DIR[0]+EN[EN_opt]+"/"
#R enatiomer ph. en. 25 bins
loc_short="angular_distr_el/"+PATH_CH[PATH_CH_opt]+DIR[1]+EN[EN_opt]+"/"
#S enatiomer ph. en. 35 bins
loc_long="angular_distr_el/"+PATH_CH[PATH_CH_opt]+DIR[2]+EN[EN_opt]+"/"

In [ ]:
#commands for uproot single graphs loading
fileR[loc_short+"R/"+ID_SHORT[0]].all_members
# fileR[loc_short+"R/"+ID_SHORT[0]].member("fXaxis") #just the object
fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").centers() #N
# fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").edges() #N+1
# fileR[loc_short+"R/"+ID_SHORT[0]].to_numpy()

In [ ]:
############################################# R enantiomer = SHORT RUN 25 bins #########################################################
#cos(theta) vs energy
ID_sel=1 #0 normal, 1 pol plane, 2 redpphi, 3 redphipolplane
Ren=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].axis("x").centers() #N
ctheta=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].axis("y").centers() #N

RcthetaEN_R=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].to_numpy()[0]
RcthetaEN_R_err=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].errors() #to be normalised too
RcthetaEN_R_norm=fugi.normalise_with_err(RcthetaEN_R,normtype=0)

RcthetaEN_L=fileR[loc_short+"L/"+ID_SHORT[ID_sel]].to_numpy()[0]
RcthetaEN_L_err=fileR[loc_short+"L/"+ID_SHORT[ID_sel]].errors() #to be normalised too
RcthetaEN_L_norm=fugi.normalise_with_err(RcthetaEN_L,normtype=0)

print("shape RcthetaEN_R = ",RcthetaEN_R.shape)
print("shape ctheta = ",ctheta.shape)
print("shape Ren = ",Ren.shape)

#PECD between two diffrent helicities
#NOTE the first energy is empty
RcPECD_en =  np.divide(np.subtract(RcthetaEN_R_norm[1:],RcthetaEN_L_norm[1:]),np.add(RcthetaEN_R_norm[1:],RcthetaEN_L_norm[1:]))
RerrcPECD_en = fugi.error_calc(RcthetaEN_R[1:],RcthetaEN_L[1:])

#533-534eV classic PECD diatomic
# standard 0-1
# redPHI 2-3
cbeta=fileR[loc_short+"R/"+ID_PECD[2]].axis("x").centers() #N
cthetaM=fileR[loc_short+"R/"+ID_PECD[2]].axis("y").centers() #N

RMFPAD_533_R=fileR[loc_short+"R/"+ID_PECD[2]].to_numpy()[0]
RMFPAD_533_R_err=fileR[loc_short+"R/"+ID_PECD[2]].errors() #to be normalised too
RMFPAD_533_R_norm=fugi.normalise_with_err(RMFPAD_533_R,normtype=0)
Rctheta_533_R=np.sum(RMFPAD_533_R, axis=1)
Rctheta_533_R_err=np.sum(RMFPAD_533_R_err, axis=1)
Rctheta_533_R_norm = fugi.normalise_with_err(Rctheta_533_R)


RMFPAD_534_R=fileR[loc_short+"R/"+ID_PECD[3]].to_numpy()[0]
RMFPAD_534_R_err=fileR[loc_short+"R/"+ID_PECD[3]].errors() #to be normalised too
RMFPAD_534_R_norm=fugi.normalise_with_err(RMFPAD_534_R,normtype=0)
Rctheta_534_R=np.sum(RMFPAD_534_R, axis=1)
Rctheta_534_R_err=np.sum(RMFPAD_534_R_err, axis=1)
Rctheta_534_R_norm = fugi.normalise_with_err(Rctheta_534_R)

RMFPAD_533_L=fileR[loc_short+"L/"+ID_PECD[2]].to_numpy()[0]
RMFPAD_533_L_err=fileR[loc_short+"L/"+ID_PECD[2]].errors() #to be normalised too
RMFPAD_533_L_norm=fugi.normalise_with_err(RMFPAD_533_L,normtype=0)
Rctheta_533_L=np.sum(RMFPAD_533_L, axis=1)
Rctheta_533_L_err=np.sum(RMFPAD_533_L_err, axis=1)
Rctheta_533_L_norm = fugi.normalise_with_err(Rctheta_533_L)

RMFPAD_534_L=fileR[loc_short+"L/"+ID_PECD[3]].to_numpy()[0]
RMFPAD_534_L_err=fileR[loc_short+"L/"+ID_PECD[3]].errors() #to be normalised too
RMFPAD_534_L_norm=fugi.normalise_with_err(RMFPAD_534_L,normtype=0)
Rctheta_534_L=np.sum(RMFPAD_534_L, axis=1)
Rctheta_534_L_err=np.sum(RMFPAD_534_L_err, axis=1)
Rctheta_534_L_norm = fugi.normalise_with_err(Rctheta_534_L)

print("shape RMFPAD_533_R = ",RMFPAD_533_R.shape)
print("shape Rctheta_533_R = ",Rctheta_533_R_norm.shape)

#PECD between two diffrent helicities
RPECD_Auger_533 =  np.divide(np.subtract(RMFPAD_533_R_norm,RMFPAD_533_L_norm),np.add(RMFPAD_533_R_norm,RMFPAD_533_L_norm))
RerrPECD_Auger_533 = fugi.error_calc(RMFPAD_533_R,RMFPAD_533_L)

RPECD_Auger_534 =  np.divide(np.subtract(RMFPAD_534_R_norm,RMFPAD_534_L_norm),np.add(RMFPAD_534_R_norm,RMFPAD_534_L_norm))
RerrPECD_Auger_534 = fugi.error_calc(RMFPAD_534_R,RMFPAD_534_L)

#PECD cos projection
RcPECD_533 =  np.divide(np.subtract(Rctheta_533_R_norm,Rctheta_533_L_norm),np.add(Rctheta_533_R_norm,Rctheta_533_L_norm))
RerrcPECD_533 = fugi.error_calc(Rctheta_533_R,Rctheta_533_L)

RcPECD_534 =  np.divide(np.subtract(Rctheta_534_R_norm,Rctheta_534_L_norm),np.add(Rctheta_534_R_norm,Rctheta_534_L_norm))
RerrcPECD_534 = fugi.error_calc(Rctheta_534_R,Rctheta_534_L)

In [ ]:
############################################# S enantiomer = SHORT RUN 35 bins #########################################################
#cos(theta) vs energy
ID_sel=1 #0 normal, 1 pol plane, 2 redpphi, 3 redphipolplane
Sen=fileS[loc_long+"R/"+ID_SHORT[ID_sel]].axis("x").centers() #N
ctheta=fileS[loc_long+"R/"+ID_SHORT[ID_sel]].axis("y").centers() #N

ScthetaEN_R=fileS[loc_long+"R/"+ID_SHORT[ID_sel]].to_numpy()[0]
ScthetaEN_R_err=fileS[loc_long+"R/"+ID_SHORT[ID_sel]].errors() #to be normalised too
ScthetaEN_R_norm=fugi.normalise_with_err(ScthetaEN_R,normtype=0)

ScthetaEN_L=fileS[loc_long+"L/"+ID_SHORT[ID_sel]].to_numpy()[0]
ScthetaEN_L_err=fileS[loc_long+"L/"+ID_SHORT[ID_sel]].errors() #to be normalised too
ScthetaEN_L_norm=fugi.normalise_with_err(ScthetaEN_L,normtype=0)

print("shape ScthetaEN_R = ",ScthetaEN_R.shape)
print("shape ctheta = ",ctheta.shape)
print("shape Sen = ",Sen.shape)

#PECD between two diffrent helicities
#NOTE the first energy is empty
ScPECD_en =  np.divide(np.subtract(ScthetaEN_R_norm,ScthetaEN_L_norm),np.add(ScthetaEN_R_norm,ScthetaEN_L_norm))
SerrcPECD_en = fugi.error_calc(ScthetaEN_R,ScthetaEN_L_norm)

#533-534eV classic PECD diatomic
# standard 0-1
# redPHI 2-3
cbeta=fileS[loc_long+"R/"+ID_PECD[2]].axis("x").centers() #N
cthetaM=fileS[loc_long+"R/"+ID_PECD[2]].axis("y").centers() #N

SMFPAD_533_R=fileS[loc_long+"R/"+ID_PECD[2]].to_numpy()[0]
SMFPAD_533_R_err=fileS[loc_long+"R/"+ID_PECD[2]].errors() #to be normalised too
SMFPAD_533_R_norm=fugi.normalise_with_err(SMFPAD_533_R,normtype=0)
Sctheta_533_R=np.sum(SMFPAD_533_R, axis=1)
Sctheta_533_R_err=np.sum(SMFPAD_533_R_err, axis=1)
Sctheta_533_R_norm = fugi.normalise_with_err(Sctheta_533_R)


SMFPAD_534_R=fileS[loc_long+"R/"+ID_PECD[3]].to_numpy()[0]
SMFPAD_534_R_err=fileS[loc_long+"R/"+ID_PECD[3]].errors() #to be normalised too
SMFPAD_534_R_norm=fugi.normalise_with_err(SMFPAD_534_R,normtype=0)
Sctheta_534_R=np.sum(SMFPAD_534_R, axis=1)
Sctheta_534_R_err=np.sum(SMFPAD_534_R_err, axis=1)
Sctheta_534_R_norm = fugi.normalise_with_err(Sctheta_534_R)

SMFPAD_533_L=fileS[loc_long+"L/"+ID_PECD[2]].to_numpy()[0]
SMFPAD_533_L_err=fileS[loc_long+"L/"+ID_PECD[2]].errors() #to be normalised too
SMFPAD_533_L_norm=fugi.normalise_with_err(SMFPAD_533_L,normtype=0)
Sctheta_533_L=np.sum(SMFPAD_533_L, axis=1)
Sctheta_533_L_err=np.sum(SMFPAD_533_L_err, axis=1)
Sctheta_533_L_norm = fugi.normalise_with_err(Sctheta_533_L)

SMFPAD_534_L=fileS[loc_long+"L/"+ID_PECD[3]].to_numpy()[0]
SMFPAD_534_L_err=fileS[loc_long+"L/"+ID_PECD[3]].errors() #to be normalised too
SMFPAD_534_L_norm=fugi.normalise_with_err(SMFPAD_534_L,normtype=0)
Sctheta_534_L=np.sum(SMFPAD_534_L, axis=1)
Sctheta_534_L_err=np.sum(SMFPAD_534_L_err, axis=1)
Sctheta_534_L_norm = fugi.normalise_with_err(Sctheta_534_L)

print("shape SMFPAD_533_R = ",SMFPAD_533_R.shape)
print("shape SMFPAD_533_R = ",Sctheta_533_R_norm.shape)

#PECD between two diffrent helicities
SPECD_Auger_533 =  np.divide(np.subtract(SMFPAD_533_R_norm,SMFPAD_533_L_norm),np.add(SMFPAD_533_R_norm,SMFPAD_533_L_norm))
SerrPECD_Auger_533 = fugi.error_calc(SMFPAD_533_R,SMFPAD_533_L)

SPECD_Auger_534 =  np.divide(np.subtract(SMFPAD_534_R_norm,SMFPAD_534_L_norm),np.add(SMFPAD_534_R_norm,SMFPAD_534_L_norm))
SerrPECD_Auger_534 = fugi.error_calc(SMFPAD_534_R,SMFPAD_534_L)

#PECD cos projection
ScPECD_533 =  np.divide(np.subtract(Sctheta_533_R_norm,Sctheta_533_L_norm),np.add(Sctheta_533_R_norm,Sctheta_533_L_norm))
SerrcPECD_533 = fugi.error_calc(Sctheta_533_R,Sctheta_533_L)

ScPECD_534 =  np.divide(np.subtract(Sctheta_534_R_norm,Sctheta_534_L_norm),np.add(Sctheta_534_R_norm,Sctheta_534_L_norm))
SerrcPECD_534 = fugi.error_calc(Sctheta_534_R,Sctheta_534_L)

In [ ]:
# ############################################# PECD R-S for specific energies ##############################################
# RMFPAD_533_norm =  RMFPAD_533_R_norm+np.flip(RMFPAD_533_L_norm) #AXIS!!!
# SMFPAD_533_norm =  SMFPAD_533_R_norm+np.flip(SMFPAD_533_L_norm) #AXIS!!!
# PECD_Auger_533 =  np.divide(np.subtract(RMFPAD_533_norm,SMFPAD_533_norm),np.add(RMFPAD_533_norm,SMFPAD_533_norm))
# errPECD_Auger_533 = fugi.error_calc(SMFPAD_533_R_norm,SMFPAD_533_L_norm)

# RMFPAD_534_norm =  RMFPAD_534_R_norm+np.flip(RMFPAD_534_L_norm) #AXIS!!!
# SMFPAD_534_norm =  SMFPAD_534_R_norm+np.flip(SMFPAD_534_L_norm) #AXIS!!!
# PECD_Auger_534 =  np.divide(np.subtract(RMFPAD_534_norm,SMFPAD_534_norm),np.add(RMFPAD_534_norm,SMFPAD_534_norm))
# errPECD_Auger_534 = fugi.error_calc(SMFPAD_534_R_norm,SMFPAD_534_L_norm)

#PECD cos projection
Sctheta_533_norm = Sctheta_533_R_norm + np.flip(Sctheta_533_L_norm)
Rctheta_533_norm = Rctheta_533_R_norm + np.flip(Rctheta_533_L_norm)
cPECD_533 =  np.divide(np.subtract(Rctheta_533_norm,Sctheta_533_norm),np.add(Rctheta_533_norm,Sctheta_533_norm))
errcPECD_533 = fugi.error_calc(Rctheta_533_R + np.flip(Rctheta_533_L),Sctheta_533_R + np.flip(Sctheta_533_L))

Sctheta_534_norm = Sctheta_534_R_norm + np.flip(Sctheta_534_L_norm)
Rctheta_534_norm = Rctheta_534_R_norm + np.flip(Rctheta_534_L_norm)
cPECD_534 =  np.divide(np.subtract(Rctheta_534_norm,Sctheta_534_norm),np.add(Rctheta_534_norm,Sctheta_534_norm))
errcPECD_534 = fugi.error_calc(Rctheta_534_R + np.flip(Rctheta_534_L),Sctheta_534_R + np.flip(Sctheta_534_L))

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
plt.suptitle(PATH_name+" cos(tehta) R enat. vs ph. en. @ el. en. 25 steps "+EN[EN_opt]+" eV")
cs1=ax1.contourf(Ren[1:],ctheta,RcthetaEN_R_norm[1:].T, cmap=cmap_temp, levels=15)
ax1.set_title("CR pol.")
ax1.set_xlabel('photon en [eV]')
ax1.set_ylabel('cos(theta) [adm]')
# ax1.set_aspect(aspect)
cbar1=fig.colorbar(cs1, shrink=0.9, ax=ax1)
cbar1.set_ticklabels(["min","max"])
cbar1.set_ticks([RcthetaEN_R_norm[1:].min(),RcthetaEN_R_norm[1:].max()])


cs2=ax2.contourf(Ren[1:],ctheta,RcthetaEN_L_norm[1:].T, cmap=cmap_temp, levels=15)
ax2.set_title("CL pol.")
ax2.set_xlabel('ph. ennergy [eV]')
ax2.set_ylabel('cos(theta) [adm]')
# ax2.set_aspect(aspect)
cbar2=fig.colorbar(cs2, shrink=0.9, ax=ax2)
cbar2.set_ticklabels(["min","max"])
cbar2.set_ticks([RcthetaEN_L_norm[1:].min(),RcthetaEN_L_norm[1:].max()])

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_cos(theta)_R_CR-CL.png", dpi=144, transparent=False)
# plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
plt.suptitle(PATH_name+" cos(tehta) S enat. vs ph. en. @ el. en. 35 steps "+EN[EN_opt]+" eV")
cs1=ax1.contourf(Sen[1:],ctheta,ScthetaEN_R_norm[1:].T, cmap=cmap_temp, levels=15)
ax1.set_title("CR pol.")
ax1.set_xlabel('photon en [eV]')
ax1.set_ylabel('cos(theta) [adm]')
# ax1.set_aspect(aspect)
cbar1=fig.colorbar(cs1, shrink=0.9, ax=ax1)
cbar1.set_ticklabels(["min","max"])
cbar1.set_ticks([ScthetaEN_R_norm[1:].min(),ScthetaEN_R_norm[1:].max()])


cs2=ax2.contourf(Sen[1:],ctheta,ScthetaEN_L_norm[1:].T, cmap=cmap_temp, levels=15)
ax2.set_title("CL pol.")
ax2.set_xlabel('ph. ennergy [eV]')
ax2.set_ylabel('cos(theta) [adm]')
# ax2.set_aspect(aspect)
cbar2=fig.colorbar(cs2, shrink=0.9, ax=ax2)
cbar2.set_ticklabels(["min","max"])
cbar2.set_ticks([ScthetaEN_L_norm[1:].min(),ScthetaEN_L_norm[1:].max()])

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_cos(theta)_S_CR-CL.png", dpi=144, transparent=False)
# plt.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(Ren[1:],ctheta,RcPECD_en.T, cmap=cmap_temp, levels=10)
ax.set_title(PATH_name+" PECD CR-CL \n R enan. vs ph. en. @ "+EN[EN_opt]+" eV")
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('photon en [eV]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
cbar.ax.set_ylabel('% of asymmetry')
# ax.set_aspect(aspect)
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECDmap_R.png", dpi=144, transparent=False)


In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(Sen[1:],ctheta,ScPECD_en[1:].T, cmap=cmap_temp, levels=10)
ax.set_title(PATH_name+" PECD CR-CL \n S enan. vs ph. en. @ "+EN[EN_opt]+" eV")
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('photon en [eV]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
cbar.ax.set_ylabel('% of asymmetry')
# ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECDmap_S.png", dpi=144, transparent=False)

In [ ]:
import functions_minuit as fumi
#NOTE adjust the subplot according to the len(en)!
# alternative nparray initialization
param_matrix_R = [[0 for i in range(6)] for j in range(len(RcPECD_en))] #i col, j row

# fig, axes = plt.subplots(4,6, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(4,6, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2) #custom_ylim = (-1,1)

# limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

for i,el,el_err,ax in zip(range(len(RcPECD_en)),RcPECD_en,RerrcPECD_en,axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    
    x_data = ctheta;y_data = el;y_err = el_err
    
    m=fumi.iminuit_err(x_data,y_data,y_err,limits)
    fumi.plotminuit(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_R[i][counter]=((m.values[el], m.errors[el]))

param_matrix_R=np.array(param_matrix_R) #numpy tranformation!!!
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_6slices_b1b2_R.png", dpi=144, transparent=False)

# fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
plt.show() 

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10))
plt.suptitle(PATH_name+" PECD6 b1 and b2 R enant. vs ph. energy @ "+EN[EN_opt]+" eV")
ax1.scatter(Ren[1:],param_matrix_R[:,0,0])
ax1.errorbar(Ren[1:],param_matrix_R[:,0,0], param_matrix_R[:,0,1], fmt="o")
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.7)
ax1.set_xlabel('ph. ennergy [eV]')
ax1.set_ylim(-0.15,0.15)
ax1.set_ylabel('b1 [adm]')
# ax1.set_aspect(aspect)

ax2.scatter(Ren[1:],param_matrix_R[:,1,0])
ax2.errorbar(Ren[1:],param_matrix_R[:,1,0], param_matrix_R[:,1,1], fmt="o")
ax2.axhline(y=-1, color='r', linestyle='--', alpha=0.7)
ax2.axhline(y=2, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('ph. ennergy [eV]')
ax2.set_ylabel('b2 [adm]')
# ax2.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_6_b1b2_R.png", dpi=144, transparent=False)
# plt.show()

In [ ]:
import functions_minuit as fumi
#NOTE adjust the subplot according to the len(en)!
# alternative nparray initialization
param_matrix_R = [[0 for i in range(2)] for j in range(len(RcPECD_en))] #i col, j row

# fig, axes = plt.subplots(4,6, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(4,6, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2) #custom_ylim = (-1,1)

limits=[(-1, 1), (-1, 2)]

for i,el,el_err,ax in zip(range(len(RcPECD_en)),RcPECD_en,RerrcPECD_en,axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    
    x_data = ctheta;y_data = el;y_err = el_err
    
    m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
    fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_R[i][counter]=((m.values[el], m.errors[el]))

param_matrix_R=np.array(param_matrix_R) #numpy tranformation!!!

# fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_2slices_b1b2_R.png", dpi=144, transparent=False)
plt.show() 

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10))
plt.suptitle(PATH_name+" b1 and b2 R enant. vs ph. energy 25 steps @ "+EN[EN_opt]+" eV")
ax1.scatter(Ren[1:],param_matrix_R[:,0,0])
ax1.errorbar(Ren[1:],param_matrix_R[:,0,0], param_matrix_R[:,0,1], fmt="o")
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.7)
ax1.set_xlabel('ph. ennergy [eV]')
ax1.set_ylim(-0.15,0.15)
ax1.set_ylabel('b1 [adm]')
# ax1.set_aspect(aspect)

ax2.scatter(Ren[1:],param_matrix_R[:,1,0])
ax2.errorbar(Ren[1:],param_matrix_R[:,1,0], param_matrix_R[:,1,1], fmt="o")
ax2.axhline(y=-1, color='r', linestyle='--', alpha=0.7)
ax2.axhline(y=2, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('ph. ennergy [eV]')
ax2.set_ylabel('b2 [adm]')
# ax2.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_b1b2_R.png", dpi=144, transparent=False)
# plt.show()

In [ ]:
import functions_minuit as fumi
#NOTE adjust the subplot according to the len(en)!
# alternative nparray initialization
param_matrix_S = [[0 for i in range(2)] for j in range(len(ScPECD_en))] #i col, j row

# fig, axes = plt.subplots(4,6, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
# fig, axes = plt.subplots(5,7, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2) #custom_ylim = (-1,1)

# limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
# limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]
limits=[(-1, 1), (-1, 2)]

# for i,el,el_err,ax in zip(range(len(ScPECD_en)),ScPECD_en,SerrcPECD_en,axes.flatten()): #fixed because of the photon directions, equal to range(len(cosphi))
for i,el,el_err in zip(range(len(ScPECD_en)),ScPECD_en,SerrcPECD_en): #no plotting version
    
    x_data = ctheta;y_data = el;y_err = el_err
    
    m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
    # fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_S[i][counter]=((m.values[el], m.errors[el]))

param_matrix_S=np.array(param_matrix_S) #numpy tranformation!!!

# fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
# plt.show()

fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10))
plt.suptitle(PATH_name+" b1 and b2 S enant. vs ph. energy 35 steps @ "+EN[EN_opt]+" eV")
ax1.scatter(Sen,param_matrix_S[:,0,0])
ax1.errorbar(Sen,param_matrix_S[:,0,0], param_matrix_S[:,0,1], fmt="o")
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.7)
ax1.set_xlabel('ph. ennergy [eV]')
ax1.set_ylim(-0.15,0.15)
ax1.set_ylabel('b1 [adm]')
# ax1.set_aspect(aspect)

ax2.scatter(Sen,param_matrix_S[:,1,0])
ax2.errorbar(Sen,param_matrix_S[:,1,0], param_matrix_S[:,1,1], fmt="o")
ax2.axhline(y=-1, color='r', linestyle='--', alpha=0.7)
ax2.axhline(y=2, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('ph. ennergy [eV]')
ax2.set_ylabel('b2 [adm]')
# ax2.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_b1b2_S.png", dpi=144, transparent=False)
# plt.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(cbeta,cthetaM,fugi.smoothgauss(RPECD_Auger_533.T,0.3,0.3), cmap=cmap_temp, levels=10)
ax.set_title(PATH_name+" PECD map CR-CL \n R ph. 533 eV @ "+EN[EN_opt]+" eV")
aspect=(cbeta.max()/cthetaM.max())
ax.set_xlabel('cos(beta) [adm]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.ax.set_ylabel('% of asymmetry')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/ph533eV_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECD_R.png", dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = cthetaM;y_data = Rctheta_533_R;y_err = Rctheta_533_R_err

m=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m,ax,custom_xlim)
ax.set_title(PATH_name+" cos(theta)_CR \n R enat. ph. 533 eV @ "+EN[EN_opt]+" eV")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax.set_aspect(aspect)

m
# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_combi_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
#PECD INTERPOLATION RANDOM
fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.1, 0.1)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = cthetaM;y_data = RcPECD_533;y_err = RerrcPECD_533

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(PATH_name+" PECD CR-CL \n R enat. ph. 533 eV @ "+EN[EN_opt]+" eV")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('PECD %')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/ph533eV_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_1DPECD_R.png", dpi=144, transparent=False)


In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(cbeta,cthetaM,fugi.smoothgauss(RPECD_Auger_534.T,0.3,0.3), cmap=cmap_temp, levels=10)
ax.set_title(PATH_name+" PECD map CR-CL \n R enat. ph. 534 eV @ "+EN[EN_opt]+" eV")
aspect=(cbeta.max()/cthetaM.max())
ax.set_xlabel('cos(beta) [adm]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
cbar.ax.set_ylabel('% of asymmetry')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/ph534eV_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECD_R.png", dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
#COMBINED ELLICITIES
import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = cthetaM;y_data = Rctheta_534_R;y_err = Rctheta_534_R_err

m=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m,ax,custom_xlim)
ax.set_title(PATH_name+" cos(theta)_CR \n R enat. ph. 534 eV @ "+EN[EN_opt]+" eV")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('normalised counts')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax.set_aspect(aspect)

m
# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_combi_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
#PECD INTERPOLATION RANDOM
fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.1, 0.1)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = cthetaM;y_data = RcPECD_534;y_err = RerrcPECD_534

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(PATH_name+" PECD CR-CL \n R enat. ph. 534 eV @ "+EN[EN_opt]+" eV")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('PECD %')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/ph534eV_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_1DPECD_R.png", dpi=144, transparent=False)


In [ ]:
#PECD INTERPOLATION RANDOM
fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.1, 0.1)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = cthetaM;y_data = cPECD_534;y_err = errcPECD_534

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(PATH_name+" PECD R-S \n ph. 534 eV @ "+EN[EN_opt]+" eV")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('PECD %')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/ph534eV_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_1DPECD_R-S.png", dpi=144, transparent=False)


In [ ]:
#PECD INTERPOLATION RANDOM
fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.1, 0.1)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = cthetaM;y_data = cPECD_533;y_err = errcPECD_533

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(PATH_name+" PECD R-S \n ph. 533 eV @ "+EN[EN_opt]+" eV")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('PECD %')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m
fig.savefig("../PYTHON_graphs/OUTPUTS/Auger/ph533eV_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_1DPECD_R-S.png", dpi=144, transparent=False)